In [1]:
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

import random

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True

In [2]:
# function to extract features from the date column
def extract_features(train, test):
    store_item_table = pd.pivot_table(train, index='store', columns='item',values='sales', aggfunc=np.mean)
    # training only on first 3 months, and year >= 2015 seem to give better scores and improve training time
    train = train[(train.index.month <= 3) & (train.index.year >= 2015)].copy()
    train['base_sales'] = train.apply(lambda row: store_item_table.at[row['store'], row['item']], axis=1)
    train['month'] = train.index.month
    train['dayofweek'] = train.index.dayofweek
    train['year'] = train.index.year - 2013
    train = train.drop(columns=['store', 'item'])

    test = test.copy()
    test['base_sales'] = test.apply(lambda row: store_item_table.at[row['store'], row['item']], axis=1)
    test['month'] = test.index.month
    test['dayofweek'] = test.index.dayofweek
    test['year'] = test.index.year - 2013
    test = test.drop(columns=['store', 'item'])

    return train, test

In [3]:
train = pd.read_csv('train.csv', parse_dates=['date'], index_col='date')
test = pd.read_csv('test.csv', parse_dates=['date'], index_col='date')
train, test = extract_features(train, test)

X_train = torch.Tensor(train.drop(columns='sales').to_numpy())
y_train = torch.Tensor(train['sales'].to_numpy())
X_test = torch.Tensor(test.drop(columns='id').to_numpy())

In [4]:
# simple feedforward neural network with single hidden layer
class FNN(nn.Module):
    def __init__(self, no_features):
        super().__init__()
        self.mlp_stack = nn.Sequential(
            nn.Linear(no_features, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
        )

    def forward(self, x):
        return self.mlp_stack(x)

def train_model(model, X_train, y_train, no_epochs):
    loss_fn = nn.L1Loss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99)
    train_dataloader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=False)
    model.train()
    for epoch in range(1, no_epochs + 1):
        running_loss = 0
        for X, y in train_dataloader:
            optimizer.zero_grad()
            output = model(X)
            loss = loss_fn(output.flatten(), y)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        scheduler.step()
        print('epoch', epoch, 'train loss:', running_loss/len(train_dataloader))

In [5]:
model = FNN(no_features=len(X_train[0]))
train_model(model, X_train, y_train, 200)

epoch 1 train loss: 7.425235985809052
epoch 2 train loss: 6.650997514202417
epoch 3 train loss: 6.289485226496984
epoch 4 train loss: 6.007922284776473
epoch 5 train loss: 5.798840124501273
epoch 6 train loss: 5.660584943602961
epoch 7 train loss: 5.56095058125522
epoch 8 train loss: 5.485168592450301
epoch 9 train loss: 5.429094110377225
epoch 10 train loss: 5.38430643847126
epoch 11 train loss: 5.344250658866479
epoch 12 train loss: 5.3029209919083895
epoch 13 train loss: 5.267967518035594
epoch 14 train loss: 5.247787625526234
epoch 15 train loss: 5.215984359566506
epoch 16 train loss: 5.1997444710718
epoch 17 train loss: 5.17918029970668
epoch 18 train loss: 5.169879567859531
epoch 19 train loss: 5.161112534200616
epoch 20 train loss: 5.155601841982408
epoch 21 train loss: 5.1502312496093445
epoch 22 train loss: 5.1548714199867645
epoch 23 train loss: 5.147741777475877
epoch 24 train loss: 5.148077875741593
epoch 25 train loss: 5.138859799495838
epoch 26 train loss: 5.1441991401236

In [6]:
with torch.no_grad():
    model.eval()
    predictions = model(X_test).flatten().numpy()

submission = pd.DataFrame({'id': test['id'], 'sales': predictions.round(0).astype('int')}).reset_index(drop=True)
submission.to_csv('submission.csv', index=False)
submission

,id,sales
0,0,12
1,1,15
2,2,15
3,3,16
4,4,17
...,...,...
44995,44995,71
44996,44996,72
44997,44997,76
44998,44998,81
